In [4]:
from IPython.core.display import HTML
import jieba


class Searcher_scan():
    def __init__(self, title_file):
        with open(title_file, 'r') as f:
            titles = f.read()
        self.title_list = list(set(titles.split())) ##转换成ji

    def word_match(self, words, title):
        ifmatch = True
        keyword_ = ' '.join(jieba.cut(words))
        for word in keyword_.split():
            if word != ' ' and word not in title:
                ifmatch = False
        return ifmatch

    def conv_query(self, query):
        query_new_parts = []
        for part in list(jieba.cut(query)):
            if part == '(' or part == ')':
                query_new_parts.append(part)
            elif part in ('and', 'AND', 'or', 'OR', 'NOT', 'not', ' '):
                query_new_parts.append(part.lower())
            else:
                query_new_parts.append("self.word_match('{}',title)".format(part))
        query_new = ''.join(query_new_parts)
        return query_new

    def highlighter(self, doc, word):
        for part in list(jieba.cut(word)):
            if part not in ('(', ')', 'and', 'AND', 'or', 'OR', 'NOT', 'not', ' '):
                doc = doc.replace(part, '<span style="color:red">{}</span>'.format(part))
        return doc

    def search(self, query):
        query_new = self.conv_query(query)
        for title in self.title_list:
            if eval(query_new):
                display(HTML(self.highlighter(title, query)))


query = '苹果 and (芯片 or 高通)'
searcher = Searcher_scan('tiles.txt')
searcher.search(query)

In [5]:
from IPython.core.display import HTML
import jieba

query = '苹果 and (芯片 or 高通)'
searcher = Searcher_scan('tiles.txt')
searcher.search(query) 

with open('tiles.txt', 'r') as f:
    titles = f.read()

index = dict()    
idx = 0
title_list = []

for title in titles.split():
    title_list.append(title)
    for term in list(jieba.cut(title)):
        if term in index:
            index[term].add(idx)
        else:
            index[term] = set([idx])
    idx += 1

query = '苹果手机'
result = None
for term in list(jieba.cut(query)):
    if result is None:
        result = index[term]
    else:
        result = result & index[term]
for docid in result:
    print(title_list[docid])